# RNNs in Tensorflow, a Practical Guide and Undocumented Features – WildML

原文 http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/<br>
My ynote: "RNNs in Tensorflow, a Practical Guide and Undocumented Features – WildML"

[x] 我懷疑這個 tf.train.SequenceExample 既然是 undocument 會不會已經 obsoleted 了？ 好像沒有。
https://www.tensorflow.org/api_docs/python/tf/train/SequenceExample
沒有內容，先跳過，免得白花功夫。就我剛才研究，反正就是把 data 轉成 `<class 'tensorflow.core.example.example_pb2.SequenceExample'>` 而得到文章中所提到的種種好處。 

[x] 本例子只是介紹怎麼 write, 怎麼 parse, 沒有介紹怎麼 read 回來。我想這一系列文章應該會有例子吧？


In [1]:
import peforth

In [2]:
%%f 擴充、修訂一下 peforth 的行為模式，讓它認得 jupyter notebook 下的 globals. Dot . 也改寫了，適合 jupyter notebook 學習環境使用。
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.
: . tib. ; // ( tos -- ) A better dot that also prints the entire command line
  /// For experiments that need to show both question and result.
  /// "" . prints the command line only, w/o the TOS.
: path-to-find-modules ( <path> -- ) // Add path to sys.path so "import module-name" can find the module
    CR word trim ( "path" ) py: sys.path.append(pop()) ;
code # print(nexttoken('\n')) end-code // Redefine \ command to print the comment line 


reDef unknown
reDef .


In [3]:
import tensorflow as tf
import numpy as np
import tempfile

In [4]:
sequences = [[1, 2, 3], [4, 5, 1], [1, 2]]
label_sequences = [[0, 1, 0], [1, 0, 0], [1, 1]]

In [10]:
%f none value locals // ( -- dict ) current locals 
%f *debug* tt> 
def make_example(sequence, labels):
    # The object we return
    ex = tf.train.SequenceExample()
    # A non-sequential feature of our example
    sequence_length = len(sequence)
    ex.context.feature["length"].int64_list.value.append(sequence_length)
    # Feature lists for the two sequential features of our example
    fl_tokens = ex.feature_lists.feature_list["tokens"]
    fl_labels = ex.feature_lists.feature_list["labels"]
    for token, label in zip(sequence, labels):
        fl_tokens.feature.add().int64_list.value.append(token)
        fl_labels.feature.add().int64_list.value.append(label)
    peforth.ok('11>',loc=locals(),cmd=':> [0] to locals')
    return ex


reDef locals

tt>.s
      0: ({'label': 0, 'token': 1, 'fl_labels': feature {
  int64_list {
    value: 1
  }
}
feature {
  int64_list {
    value: 0
  }
}
feature {
  int64_list {
    value: 0
  }
}
, 'fl_tokens': feature {
  int64_list {
    value: 4
  }
}
feature {
  int64_list {
    value: 5
  }
}
feature {
  int64_list {
    value: 1
  }
}
, 'sequence_length': 3, 'ex': context {
  feature {
    key: "length"
    value {
      int64_list {
        value: 3
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "labels"
    value {
      feature {
        int64_list {
          value: 1
        }
      }
      feature {
        int64_list {
          value: 0
        }
      }
      feature {
        int64_list {
          value: 0
        }
      }
    }
  }
  feature_list {
    key: "tokens"
    value {
      feature {
        int64_list {
          value: 4
        }
      }
      feature {
        int64_list {
          value: 5
        }
      }
      feature {
        i

In [15]:
# Write all examples into a TFRecords file
with tempfile.NamedTemporaryFile(delete=False) as fp:
    writer = tf.python_io.TFRecordWriter(fp.name)
    for sequence, label_sequence in zip(sequences, label_sequences):
        ex = make_example(sequence, label_sequence)
        writer.write(ex.SerializeToString())
    # %f *debug* 22> 
    writer.close()  # 好像一 close() 該 file 就不見了，被 delete 掉了，所以只在斷點時才看得見。
    print("Wrote to {}".format(fp.name))
    
# 這段 example code 沒啥用，tempfile 根本看不見，除非設斷點如上。但是要看 ex 直接看就好了。
# ex 中確實有三組 records. ===> 加上 delete=False 就不會被 delete 了。

11>exit
11>11>exit
11>11>exit
11>
22>exit
22>Wrote to C:\Users\hcche\AppData\Local\Temp\tmpoovxwq8u


In [13]:
# 照他上面這樣做，一 close() 檔案就不見了。以下改成自己指定的檔名看看。。。
filename = 'SequenceExample.data'
writer = tf.python_io.TFRecordWriter(filename)
for sequence, label_sequence in zip(sequences, label_sequences):
    ex = make_example(sequence, label_sequence)
    writer.write(ex.SerializeToString())
%f *debug* 22> 
writer.close()  # 好像一 close() 該 file 就不見了，被 delete 掉了，所以只在斷點時才看得見。
print("Wrote to {}".format(filename))

# 果然 close() 之後檔案有留下來，沒有被馬上 delete 掉。See Ynote for the above example,我有抓圖下來。
# 一查發現有個 argument delete=True <-- this is default 

11>exit
11>11>exit
11>11>exit
11>
22>exit
22>Wrote to SequenceExample.data


In [ ]:
%f ex :> SerializeToString() . 

In [ ]:
tf.reset_default_graph()

# A single serialized example
# (You can read this from a file using TFRecordReader)
ex = make_example([1, 2, 3], [0, 1, 0]).SerializeToString()

# Define how to parse the example
context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}

# Parse the example (returns a dictionary of tensors)
context_parsed, sequence_parsed = tf.parse_single_sequence_example(
    serialized=ex,
    context_features=context_features,
    sequence_features=sequence_features
)

context = tf.contrib.learn.run_n(context_parsed, n=1, feed_dict=None)
print(context[0])
sequence = tf.contrib.learn.run_n(sequence_parsed, n=1, feed_dict=None)
print(sequence[0])

In [ ]:
%f ex type .
%f ex dir .
%f ex .

In [14]:
help(tempfile)

Help on module tempfile:

NAME
    tempfile - Temporary files.

DESCRIPTION
    This module provides generic, low- and high-level interfaces for
    creating temporary files and directories.  All of the interfaces
    provided by this module can be used without fear of race conditions
    except for 'mktemp'.  'mktemp' is subject to race conditions and
    should not be used; it is provided for backward compatibility only.
    
    The default path names are returned as str.  If you supply bytes as
    input, all return values will be in bytes.  Ex:
    
        >>> tempfile.mkstemp()
        (4, '/tmp/tmptpu9nin8')
        >>> tempfile.mkdtemp(suffix=b'')
        b'/tmp/tmppbi8f0hy'
    
    This module also provides some data items to the user:
    
      TMP_MAX  - maximum number of names that will be tried before
                 giving up.
      tempdir  - If this is set to a string before the first use of
                 any routine from this module, it will be considered as
   